# 1 N-Gram的介绍
N-Gram是基于一个假设：第n个词语前n-1个词相关，而与其他任何词不相关（这也是隐马尔可夫当中的假设）。整个句子出现的概率等于各个词出现的概率乘积。各个词的概率可以通过语料中统计计算得到。通常N-Gram取自文本或语料库。

N=1时称为unigram，N=2称为bigram，N=3称为trigram，假设下一个词的出现依赖它前面的一个词，即 bigram，假设下一个词的出现依赖它前面的两个词，即 trigram，以此类推。

举例中文：“你今天休假了吗”，它的bigram依次为：

你今，今天，天休，休假，假了，了吗

理论上，n 越大越好，经验上，trigram 用的最多，尽管如此，原则上，能用 bigram 解决，绝不使用 trigram。

假设句子T是有词序列w1,w2,w3...wn组成，用公式表示N-Gram语言模型如下：

```python
P(T)=P(w1)*p(w2)*p(w3)***p(wn)=p(w1)*p(w2|w1)*p(w3|w1w2)** *p(wn|w1w2w3...) p(T) 就是语言模型，即用来计算一个句子 T 概率的模型。
```

以上公式难以实际应用。此时出现马尔可夫模型，该模型认为，一个词的出现仅仅依赖于它前面出现的几个词。这就大大简化了上述公式。

```python
P(w1)P(w2|w1)P(w3|w1w2)…P(wn|w1w2…wn-1)≈P(w1)P(w2|w1)P(w3|w2)…P(wn|wn-1)
```

一般常用的N-Gram模型是Bi-Gram和Tri-Gram。分别用公式表示如下：

```python
Bi-Gram:  P(T)=p(w1|begin)*p(w2|w1)*p(w3|w2)***p(wn|wn-1) <br>
Tri-Gram:  P(T)=p(w1|begin1,begin2)*p(w2|w1,begin1)*p(w3|w2w1)***p(wn| wn-1,wn-2)
```

注意上面概率的计算方法：P(w1|begin)=以w1为开头的所有句子/句 子总数；p(w2|w1)=w1,w2同时出现的次数/w1出现的次数。以此类推

# 2 一个经典的二元语言模型例子

| 词 | 词频 |
|----|-----|
|  I    | 3437|
|  want | 1215|
|  to   | 3256|
|  eat  | 838 |
|Chinese| 213 |
| food  | 1506|
| lunch | 459 | 

语料库中一些单词的词频，统计出各个单词与其他单词的前后联系的频次，组成一个7*7的二维矩阵，如下图

|       |I     | want |  to  |  eat | Chinese | food | lunch |
|-------|------|------|------|------|---------|------|-------|
|  I    |  8   | 1087 |  0   |  13  |    0    |  0   |   0   |
| want  |  3   | 0    |  786 |  0   |    6    |  8   |   6   |
| to    |  3   | 0    |  10  |  860 |    3    |  0   |   12  |
| eat   |  0   | 0    |  2   |  0   |    19   |  2   |   52  |
|Chinses|  2   | 0    |  0   |  0   |    0    |  120 |   1   |
| food  |  19  | 0    |  17  |  0   |    0    |  0   |   0   |
| lunch |  4   | 0    |  0   |  0   |    0    |  1   |   0   |

那么语句  “I want to eat Chinese food”  的二元语言模型概率计算过程如下


P(I eant to eat Chinses food)

= P(I) * P(want|I) * P(to|want) * P(eat|to) * P(Chinese|eat) * P(food|Chinese) * P(food|lunch)

= 0.25 * 1087/3437 * 768/1215 * 860/3256 * 19/938 * 120/213

= 0.000134171

# 3 构建N-Gram语言模型
通常，通过计算最大似然估计（Maximum Likelihood Estimate）构造语言模型，这是对训练数据的最佳估计，如 bigram 公式如下：

p(wi|wi−1)=fraccount(wi−1,wi)count(wi−1)——条件概率

如给定句子集“< s> I am Sam < / s>

                       < s > Sam I am < / s>

                       < s > I do not like green eggs and ham < / s>”

部分 bigram 语言模型如下所示

$$
P(I|<s>) = \frac{2}{3} = 0.67 \\
P(Sam|<s>) = \frac{1}{3} = 0.33 \\
P(am|I) = \frac{2}{3} = 0.67 \\
P(<s>|Sam) = \frac{1}{2} = 0.5 \\
P(Sam|am) = \frac{1}{2} = 0.67 \\
P(do|I) = \frac{1}{3} = 0.33 \\
$$

词频表示如下：

| i    | want | to   | eat | chinese | food |lunch| spend|
|------|------|------|-----|---------|------|-----|------|
| 2533 |  927 | 2417 | 746 | 158     | 1093 | 341 |  278 | 

count(wi−1,wi) 如下:

|       |I     | want |  to  |  eat | Chinese | food | lunch | spend |
|-------|------|------|------|------|---------|------|-------|-------|
|  I    |  5   | 827  |  0   |  9   |    0    |  0   |   0   |   2   | 
| want  |  2   | 0    |  608 |  1   |    6    |  6   |   5   |   1   |
| to    |  2   | 0    |  4   |  686 |    2    |  0   |   6   |   211 | 
| eat   |  0   | 0    |  2   |  0   |    16   |  2   |   42  |   0   |
|Chinses|  1   | 0    |  0   |  0   |    0    |  82  |   1   |   0   |
| food  |  15  | 0    |  15  |  0   |    1    |  4   |   0   |   0   |
| lunch |  2   | 0    |  0   |  0   |    0    |  1   |   0   |   0   |
| spend |  1   | 0    |  0   |  0   |    0    |  0   |   0   |   0   |

则 bigram 为：

|       |I      | want |  to   |  eat | Chinese | food | lunch | spend |
|-------|-------|------|-------|------|---------|------|-------|-------|
|  I    |0.002  |0.33  |0      |0.0036|0        |0     |0      |0.00079| 
| want  |0.0022 |0     |0.66   |0.0011|0.0065   |0.0065|0.0054 |0.0011 |
| to    |0.00083|0     |0.0017 |0.28  |0.00083  |0     |0.0025 |0.087  | 
| eat   |0      |0     |0.0027 |0     |0.021    |0.0027|0.056  |0      |
|Chinses|0.0063 |0     |0      |0     |0        |0.52  |0.0063 |0      |
| food  |0.014  |0     |0.014  |0     |0.00092  |0.0037|0      |0      |
| lunch |0.0059 |0     |0      |0     |0        |0.0029|0      |0      |
| spend |0.0036 |0     ||0.0036|0     |0        |0     |0      |0      |

那么，句子“< s > I want chinese food < / s >”的概率为：

p(< s > Iwantchinesefood< / s >)

=p(I|< s >)P(want|I)p(chinese|want)p(food|chinese)p(< / s>|food)=.000031